In [16]:
%pip install --upgrade --quiet transformers huggingface_hub
%pip install --upgrade --quiet pinecone-client pinecone-text pinecone-notebooks
%pip install --quiet --upgrade langchain langchain-text-splitters langchain-community
%pip install --quiet openai
%pip install --quiet pytesseract pillow pdf2image
import nltk
nltk.download('punkt_tab')

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package punkt_tab to C:\Users\Balaji
[nltk_data]     J\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [24]:
import os
import re
import pytesseract
from pdf2image import convert_from_path


from pinecone_text.sparse import BM25Encoder
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.retrievers import PineconeHybridSearchRetriever

# ✅ Load config (handles .env locally or Secrets Manager on AWS)
from config import (
    PINECONE_API_KEY, 
    PINECONE_INDEX_NAME,
    HF_TOKEN
)

In [25]:
import os
from pinecone import Pinecone 
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

HF_TOKEN = os.environ.get("HF_TOKEN")
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
PINECONE_INDEX_NAME = os.environ.get("PINECONE_INDEX_NAME")

if HF_TOKEN is None or PINECONE_API_KEY is None or PINECONE_INDEX_NAME is None:
    raise ValueError("HF_TOKEN, PINECONE_API_KEY, or PINECONE_INDEX_NAME not set.")

# Set HuggingFace token
os.environ["HF_TOKEN"] = HF_TOKEN

# Initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# Create index if it doesn't exist
if PINECONE_INDEX_NAME not in pc.list_indexes().names():
    pc.create_index(
        name=PINECONE_INDEX_NAME,
        dimension=384,
        metric="dotproduct"
    )

# Connect to the index
index = pc.Index(PINECONE_INDEX_NAME)

# HuggingFace embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [19]:
bm25_encoder = BM25Encoder().default()

In [20]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"  # change path on EC2 (Linux)

def clean_text(text: str) -> str:
    """Clean extracted text (remove extra spaces, line breaks)."""
    return " ".join(text.split())

def extract_text_using_ocr(pdf_path: str) -> str:
    """Extract text from all pages of a PDF file using OCR."""
    images = convert_from_path(
        pdf_path,
        poppler_path=r"C:\Users\Balaji J\OneDrive\Desktop\Release-25.07.0-0\poppler-25.07.0\Library\bin"  # change path on EC2 (Linux)
    )

    total_pages = len(images)
    all_text = []

    for page_num, image in enumerate(images):
        print(f"Extracting text from page {page_num + 1} of {total_pages}...")
        page_text = pytesseract.image_to_string(image)
        cleaned_page_text = clean_text(page_text)
        all_text.append(cleaned_page_text)

    return "\n".join(all_text)

In [21]:
def process_books_folder(books_folder, output_text_folder):
    """
    Loop through all PDFs in books_folder, run OCR,
    and save extracted text into output_text_folder.
    """
    if not os.path.exists(output_text_folder):
        os.makedirs(output_text_folder)

    for file_name in os.listdir(books_folder):
        if file_name.endswith(".pdf"):
            pdf_path = os.path.join(books_folder, file_name)
            print(f"Processing {pdf_path} ...")

            # Extract text
            extracted_text = extract_text_using_ocr(pdf_path)

            # Save to .txt
            txt_path = os.path.join(
                output_text_folder,
                os.path.splitext(file_name)[0] + ".txt"
            )
            with open(txt_path, "w", encoding="utf-8") as f:
                f.write(extracted_text)

            print(f"Saved: {txt_path}")

In [22]:
books_folder = r"C:\Users\Balaji J\OneDrive\Desktop\rag-veda-master\books"
output_text_folder = r"C:\Users\Balaji J\OneDrive\Desktop\rag-veda-master\extracted_text"

process_books_folder(books_folder, output_text_folder)

# --- Now load all .txt, chunk, embed, push to Pinecone ---
all_text = ""
for file_name in os.listdir(output_text_folder):
    if file_name.endswith(".txt"):
        file_path = os.path.join(output_text_folder, file_name)
        with open(file_path, "r", encoding="utf-8") as file:
            all_text += file.read()

chunk_size = 500
overlap = 50
sentences = [all_text[i:i + chunk_size] for i in range(0, len(all_text), chunk_size - overlap)]


# Fit BM25
bm25_encoder.fit(sentences)
bm25_encoder.dump("rag-veda.json")

# Reload and create retriever
bm25_encoder = BM25Encoder().load("rag-veda.json")
retriever = PineconeHybridSearchRetriever(
    embeddings=embeddings,
    sparse_encoder=bm25_encoder,
    index=index
)
retriever.add_texts(sentences)

Processing C:\Users\Balaji J\OneDrive\Desktop\rag-veda-master\books\humira.pdf ...
Extracting text from page 1 of 111...
Extracting text from page 2 of 111...
Extracting text from page 3 of 111...
Extracting text from page 4 of 111...
Extracting text from page 5 of 111...
Extracting text from page 6 of 111...
Extracting text from page 7 of 111...
Extracting text from page 8 of 111...
Extracting text from page 9 of 111...
Extracting text from page 10 of 111...
Extracting text from page 11 of 111...
Extracting text from page 12 of 111...
Extracting text from page 13 of 111...
Extracting text from page 14 of 111...
Extracting text from page 15 of 111...
Extracting text from page 16 of 111...
Extracting text from page 17 of 111...
Extracting text from page 18 of 111...
Extracting text from page 19 of 111...
Extracting text from page 20 of 111...
Extracting text from page 21 of 111...
Extracting text from page 22 of 111...
Extracting text from page 23 of 111...
Extracting text from page 24 

100%|██████████| 42/42 [01:31<00:00,  2.18s/it]


In [23]:
# Make sure index_name exists
index_name = PINECONE_INDEX_NAME
index = pc.Index(index_name)
index  # now this will work


Converting the Pdfs into text format